In [1]:
from decouple import config
from binance.client import Client
import pandas as pd
import pandas_ta as ta
import json
import os
import datetime
import time

/Users/archit/Algo/algoenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
client = Client(config('API_KEY'), config('API_SECRET'), testnet=True)

In [3]:
balance = client.get_asset_balance(asset='BTC')
asset = "BTCUSDT"
balance

{'asset': 'BTC', 'free': '1.01300000', 'locked': '0.00000000'}

In [4]:
def fetch_klines(asset):
    klines = client.get_historical_klines(asset, Client.KLINE_INTERVAL_1MINUTE, "1 hour ago UTC")
    # first item is kline open timestamp, fifth item is the close price (index 0 and 4)
    klines = [[x[0], float(x[4])] for x in klines]
    klines = pd.DataFrame(klines, columns=['time', 'price'])
    klines['time'] = pd.to_datetime(klines['time'], unit='ms')
    return klines

btc = fetch_klines(asset)
btc.head()
    

,time,price
0,2024-08-25 10:59:00,63881.01
1,2024-08-25 11:00:00,63881.02
2,2024-08-25 11:01:00,63908.88
3,2024-08-25 11:02:00,63936.15
4,2024-08-25 11:03:00,63949.00


In [5]:
def get_rsi(asset): 
    klines = fetch_klines(asset)
    klines['rsi'] = ta.rsi(close = klines['price'], length=14)
    # get the most updated rsi value
    return klines["rsi"].iloc[-1]

get_rsi(asset)

35.37510200818853

In [6]:
def create_account():
    account = {
        "is_buying": True,
        "assets": {},
    }
    with open("./data/bot_account.json", "w") as f:
        f.write(json.dumps(account))

In [7]:
def log(msg):
    print(f"LOG: {msg}")
    if not os.path.exists("logs"):
        os.mkdir("logs")
    now = datetime.datetime.now()
    today = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    with open(f"logs/{today}.txt", "a+") as log_file:
        log_file.write(f"{time} - {msg}\n")

In [8]:
def trade_log(sym, side, price, amount):
    try:
        log(f"{side} {amount} {sym} for {price} per")
        if not os.path.exists("trades"):
            os.mkdir("trades")

        now = datetime.datetime.now()
        today = now.strftime("%Y-%m-%d")


        if not os.path.isfile(f"trades/{today}.csv"):
            with open(f"trades/{today}.csv", "w") as trade_file:
                trade_file.write("sym,side,amount,price\n")
        with open(f"trades/{today}.csv", "a+") as trade_file:
            trade_file.write(f"{sym},{side},{amount},{price}\n")
    except Exception as e:
        log(f"Error logging trade: {e}")

In [9]:
def trade(account, client, asset, side, quantity):
    if side == "buy":
        order = client.order_market_buy(symbol=asset, quantity=quantity)
        account["is_buying"] = False
    elif side == "sell":
        order = client.order_market_sell(symbol=asset, quantity=quantity)
        account["is_buying"] = True
    order_id = order["orderId"]
    while order["status"] != "FILLED":
        order = client.get_order(symbol=asset, orderId=order_id)
        time.sleep(1)
    print(order)
    price_paid = sum([float(fill["price"]) * float(fill["qty"]) for fill in order["fills"]])
    print(f"Price paid: {price_paid}")
    trade_log(asset, side, price_paid, quantity)

    with open("./data/bot_account.json", "w") as f:
        f.write(json.dumps(account))

In [ ]:
rsi = get_rsi(asset)
old_rsi = rsi
entry = 25
exit = 74

while True:
    try:
        if not os.path.exists("./data/bot_account.json"):
            create_account()
        with open("./data/bot_account.json", "r") as f:
            account = json.load(f)

        old_rsi = rsi
        rsi = get_rsi(asset)

        if account["is_buying"]:
            if rsi < entry and old_rsi > entry:
                trade(account, client, asset, "buy", 0.01)
            else:
                if rsi > exit and old_rsi < exit:
                    trade(account, client, asset, "sell", 0.01)
        time.sleep(10)
    except Exception as e:
        log("Error" + str(e))